In [ ]:
!pip install gpiozero

In [ ]:
import logging
from ultralytics import YOLO
import cv2
import cvzone
import math
import time
import pyttsx3
from gpiozero import DistanceSensor, AngularServo

In [ ]:
tts_engine = pyttsx3.init()

voices = tts_engine.getProperty('voices')
tts_engine.setProperty('voice', voices[1].id) 
tts_engine.setProperty('rate', 150)
tts_engine.setProperty('volume', 0.9)

In [ ]:
#To avoid YOLO from logging unnecessarily. Prevents cluttering
logging.getLogger("ultralytics").setLevel(logging.WARNING)

# url = "http://192.168.126.75:8080/video"  # mobile cam URL
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  #width
cap.set(4, 720)   #height

model = YOLO('yolo11m.pt')

#coco classes
with open("coco.names", "r") as coco_name_file:
    classNames = [line.strip() for line in coco_name_file.readlines()]

In [ ]:
# Initialize distance sensor and servo motor
ultrasonic_sensor = DistanceSensor(echo=17, trigger=4) 
servo = AngularServo(18, min_angle=-90, max_angle=90)

#object tracking to prevent re-detection within time threshold
# Set up object detection tracking
detected_objects = set()  
previously_detected = set()  
announcement_delay = 5  
last_announcement_time = time.time()  

prev_frame_time = 0
new_frame_time = 0

# Screen width for determining object position
screen_width = 1280

In [ ]:
def measure_distance(direction):
    """Rotate servo and measure distance based on the object's direction."""
    if direction == "left":
        servo.angle = -45
    elif direction == "right":
        servo.angle = 45
    else:
        servo.angle = 0  # center position

    # Small delay to allow servo to reach position
    time.sleep(0.1)
    distance = ultrasonic_sensor.distance * 100  # Convert to centimeters
    return round(distance, 1)

person with confidence 0.95
cup with confidence 0.35
cup with confidence 0.39
toothbrush with confidence 0.44
cell phone with confidence 0.63
cell phone with confidence 0.33
cell phone with confidence 0.28
cell phone with confidence 0.34
chair with confidence 0.53
cell phone with confidence 0.43


Start the camera and read objects, calculate distance through ultrasonic sensor and direction through servo motor.

In [ ]:
while True:
    new_frame_time = time.time()
    success, img = cap.read()
    if not success:
        print("Failed to capture image from camera.")
        tts_engine.say("Error, Failed to capture image from camera. Contact support")
        break
    
    results = model(img, stream=True)
    current_detected_objects = set()  # Reset current detected objects for this frame

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))

            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            class_name = classNames[cls]

            # Determine the direction of the detected object
            object_center_x = (x1 + x2) / 2
            if object_center_x < screen_width / 3:
                direction = "left"
            elif object_center_x > 2 * screen_width / 3:
                direction = "right"
            else:
                direction = "center"

            # Add to the current detected objects
            current_detected_objects.add((class_name, direction))

            display_text = f'{class_name} {conf} {direction}'
            cvzone.putTextRect(img, display_text, (max(0, x1), max(35, y1)), scale=1, thickness=1)

    # Check for new objects that have entered the frame
    for obj, direction in current_detected_objects:
        if obj not in detected_objects and time.time() - last_announcement_time >= announcement_delay:
            # Measure the distance in the specified direction
            distance = measure_distance(direction)
            announcement = f"{obj} detected on the {direction}, distance is {distance} centimeters"
            tts_engine.say(announcement)
            tts_engine.runAndWait()
            detected_objects.add(obj)
            last_announcement_time = time.time()

    # Check for objects that have exited the frame
    for obj in list(detected_objects):
        if obj not in [o[0] for o in current_detected_objects]:  # Only the class name
            previously_detected.add(obj)
            detected_objects.remove(obj)

    # Update FPS
    fps = 1 / (new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time
    print(f"FPS: {fps}")

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting program...")
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()